# **Milestone 3:**
Semantic Search with ML and BERT


In [1]:
!pip install transformers
!pip install -U sentence-transformers
!pip install elasticsearch

### **Importing the required modules**

In [2]:
# import libraries
import json
import torch
import numpy as np
from elasticsearch import Elasticsearch, helpers
from transformers import AutoModel, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer, util
from pprint import pprint

### **Getting the data**

In [3]:
DATA_DIR = '/mnt/c/Users/Daniel/elasticsearch/'

In [4]:
# load the json file
with open(DATA_DIR + '03_data.json', 'r') as outfile:
    data = json.load(outfile)

In [5]:
# print sample sentences
g = (d for d in data)
[next(g) for i in range(2)]

[{'section_title': 'Summary',
  'text': 'Pandemic,Summary\nA pandemic (from Greek πᾶν, pan, "all" and δῆμος, demos, "people") is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. A widespread endemic disease with a stable number of infected people is not a pandemic. Widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.\nThroughout human history, there have been a number of pandemics of diseases such as smallpox and tuberculosis. The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century. The term was not used yet but was for later pandemics including the 1918 influenza pandemic (Spanish flu). Current pa

In [6]:
print(len(data))

401


#### **Create Elasticsearch index**

In [7]:
# check if the elasticsearch container is running
!curl http://localhost:9200/

{
  "name" : "45b7ae5ec761",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "hZVmxIutQiGGDeXFD_KWCg",
  "version" : {
    "number" : "7.16.2",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "2b937c44140b6559905130a8650c64dbd0879cfb",
    "build_date" : "2021-12-18T19:42:46.604893745Z",
    "build_snapshot" : false,
    "lucene_version" : "8.10.1",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [8]:
# instantiate elasticsearch client
client = Elasticsearch()

In [9]:
# use helpers' bulk API to index list of elasticsearch docs
response = helpers.bulk(client, data, index='pandemics')

/home/d4niel92/miniconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [10]:
# count documents in all indices
client.cat.count("_all", params={"format": "json"})

/tmp/ipykernel_20691/3852563500.py:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  client.cat.count("_all", params={"format": "json"})


[{'epoch': '1641987825', 'timestamp': '11:43:45', 'count': '0'}]

In [11]:
# check currently available indices
client.indices.get_alias("_all")

/tmp/ipykernel_20691/966010936.py:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  client.indices.get_alias("_all")


{'pandemics': {'aliases': {}}}

In [12]:
# count documents in specific index
client.cat.count(['pandemics'], params={"format": "json"})

/tmp/ipykernel_20691/4085913714.py:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  client.cat.count(['pandemics'], params={"format": "json"})


[{'epoch': '1641987825', 'timestamp': '11:43:45', 'count': '0'}]